# 栈，队列与双端队列

## 栈

> 栈就是一种只允许在表尾进行插入和删除操作的线性表
> 例如网站列表，你能后按返回返回到前面一个网站，再比如说是文本编译器中提供的撤销机制，但是，他们都只能说是在前面和后面进行你的操作

#### 一个栈要实现以下功能

> 1. 在最上面添加元素 push
> 2. 在栈的最上面删去并返回一个元素 pop
> 3. 在不删除最上面元素的情况下得到它 top
> 4. 判断栈堆是否是空 is_empty
> 5. 返回栈中元素个数 len


In [1]:
# 在python中我们使用列表来实现这些功能

class ArrayStack:  # 这样命名强调，栈是基于数组的基础之上的数据结构
    """数据类型，栈"""

    def __init__(self):
        """创造一个列表"""
        self.data = []

    def __len__(self):
        """返回列表的长度"""
        return len(self.data)

    def is_empty(self):
        """判断列表是否为空"""
        return self.data == []

    def push(self, e):
        """在列表的最顶端添加元素"""
        self.data.append(e)

    def top(self):
        """返回最上端的元素"""
        if self.is_empty():
            raise IndexError("stack is empty")
        return self.data[-1]

    def pop(self):
        """删去并返回最上端的元素"""
        if self.is_empty():
            raise IndexError("stack is empty")  # 这里进行异常捕获
        return self.data.pop()   # 元素是被删去了的

#### 栈可以实现逆序过程

In [11]:
# 下面是一个可以把文本逆序输出的程序

def reverse_txt(filename):
    """将文件中的文本倒叙"""
    S = ArrayStack()
    origin = open(filename, encoding="utf-8") # 这里要是报错可以试试其他的编码，比如gbk
    for line in origin:
        S.push(line.strip('\n'))
        print(line)
    origin.close()

    output = open(filename, 'w', encoding="utf-8")
    while not S.is_empty():
        output.write(S.pop() + '\n')
    print(output)
    output.close()

reverse_txt('data/how_old.txt')


你今年多少岁啦

今年十八岁~~~

多少岁？

今年十八岁~~~~

<_io.TextIOWrapper name='data/how_old.txt' mode='w' encoding='utf-8'>


In [16]:
# 合理的运用栈，能准确保存你要的数据或者判断数据，精确的说是暂存你的数据
# 例：计算器中识别括号的功能
def is_matched(expr):
    """判断这个运算符是否符合计算顺序"""
    left = '([{'
    right = ')]}'
    S = ArrayStack()
    for c in expr:
        if c in left:
            S.push(c)
        elif c in right:
            if S.is_empty():
                return False
            if right.index(c) != left.index(S.pop()):
                return False
    return True
expr1 = ')[]{()}'
expr2 = '[({}])'
expr3 = '[{}]'
print(is_matched(expr1))
print(is_matched(expr2))
print(is_matched(expr3))

False
False
True


思考？这里有运用递归的必要吗

####  **总结一下的栈的特点**
> 1. 先入后出，后入先出
> 2. 除了首尾节点之外，每个元素都有前继，也有后驱
> 3. 在HTML上有大量的体现

## 队列

> 队列（queue）是只允许在一端进行插入操作，在另一端进行删除操作的线性表，简称“队”。队列是一种先进先出（First In First Out）的线性表，简称FIFO。允许插入的一端称为队尾（rear），允许删除的一端称为队头(front)。向队列中插入新的数据元素称为入队，新入队的元素就成为了队列的队尾元素。从队列中删除队头元素称为出队，其后继元素成为新的队头元素。。

#### 一个队列要实现以下功能

> 1. 在队列的末尾添加元素
> 2. 从队列移除并返回第一个元素
> 3. 在不移除第一个元素的情况下获取第一个元素
> 4. 判断队列是否为空
> 5. 返回元素个数

###### **队列与栈不同的地方在于。队列是先进后出，为了节省空间的申请。我们可以循环使用队列的空间**

In [1]:
# 在python中任然使用列表实现相关的功能

class ArrayQueue:
    """用列表实现队列"""
    DEFAULT_CAPACITY = 10

    def __init__(self):
        self._data = [None] * ArrayQueue.DEFAULT_CAPACITY
        self._size = 0
        self._front = 0  # 注意这里变量前面的下划线，说明是封装起来的变量

    def __len__(self):
        """返回队列的长度"""
        return self._size

    def is_empty(self):
        """判断这个列表是不是空集"""
        if self._size:
            return False
        return True

    def first(self):
        """
        获取但是不删去第一个元素
        :return: 第一个元素
        """
        if self.is_empty():
            raise IndexError("queue is empty")
        return self._data[self._front]

    def dequeue(self):
        """
        删去并获取第一个元素
        :return: 第一个元素
        """
        if self.is_empty():
            raise IndexError("queue is empty")
        data = self._data[self._front]
        self._data[self._front] = None
        self._front = (self._front + 1) % len(self._data) # 这里的取余很巧妙的从头开始
        self._size -= 1
        return data

    def enqueue(self, e):
        """在队列的最后一位添加元素"""
        if self._size == len(self._data):
            self.resize(len(self._data) * 2)
        avail = ( self._front + self._size ) % len(self._data)
        self._data[avail] = e
        self._size += 1

    def resize(self, capacity):
        """把队列改变容量大小"""
        old = self._data
        self._data = [None] * capacity
        walk = self._front
        for i in range(capacity):
            self._data[i] = old[walk]   # self._front 一直是一个指针一样的参量
            walk = (walk + 1) % len(old) # 这里把指针变成0，所以我们的要设置这个walk这个量
        self._front = 0


## 双端队列

> 双端队列（deque） 是一种特殊的队列数据结构，它允许在队列的两端进行插入和删除操作。与普通队列不同，普通队列只能在队尾添加元素，在队首删除元素，而双端队列可以在队首和队尾同时进行这些操作

#### 相对于队列要实现的功能

> 1. 能删除并获取获取队尾的元素
> 2. 能在队尾增加元素
> 3. 能在不删除最后一个元素的前提下得到队尾的元素

In [ ]:
# 只需要在队列的基础上稍微改写一点

class ArrayDeQueue:
    """用列表实现队列"""
    DEFAULT_CAPACITY = 10

    def __init__(self):
        self._data = [None] * ArrayDeQueue.DEFAULT_CAPACITY
        self._size = 0
        self._front = 0
        self._last = 0

    def __len__(self):
        """返回队列的长度"""
        return self._size

    def is_empty(self):
        """判断这个列表是不是空集"""
        if self._size:
            return False
        return True

    def first(self):
        """
        获取但是不删去第一个元素
        :return: 第一个元素
        """
        if self.is_empty():
            raise IndexError("queue is empty")
        return self._data[self._front]

    def last(self):
        """
        获取但不删去最后一个元素
        :return: 最后一个元素
        """
        if self.is_empty():
            raise IndexError("queue is empty")
        return self._data[self._last]

    def de_first(self):
        """
        删去并获取第一个元素
        :return: 第一个元素
        """
        if self.is_empty():
            raise IndexError("queue is empty")
        data = self._data[self._front]
        self._data[self._front] = None
        self._front = (self._front + 1) % len(self._data) # 这里的取余很巧妙的从头开始
        self._size -= 1
        return data

    def de_last(self):
        """
        获取并删去最后一个元素
        :return: 最后一个元素
        """
        if self.is_empty():
            raise IndexError("queue is empty")
        data = self._data[self._last]
        self._data[self._last] = None
        self._last = (self._last - 1) % len(self._data)
        self._size -= 1
        return data

    def en_last(self, e):
        """在队列的最后一位添加元素"""
        if self._size == len(self._data):
            self.resize(len(self._data) * 2)
        avail = ( self._front + self._size ) % len(self._data)
        self._data[avail] = e
        self._size += 1

    def en_first(self, e):
        """在队列的开头添加元素"""
        if self._size == len(self._data):
            self.resize(len(self._data) * 2)
        avail = (self._last - self._size) % len(self._data)
        self._data[avail] = e
        self._size += 1

    def resize(self, capacity):
        """把队列改变容量大小"""
        old = self._data
        self._data = [None] * capacity
        walk = self._front
        for i in range(capacity):
            self._data[i] = old[walk]   # self._front 一直是一个指针一样的参量
            walk = (walk + 1) % len(old) # 这里把指针变成0，所以我们的要设置这个walk这个量
        self._front = 0


#### **总结一下队列的特点与功能**

> 栈（Stack）和队列（Queue）是两种基本的数据结构，它们在数据的存储和访问方式上有着本质的不同。栈是一种后进先出（LIFO, Last In First Out）的结构，而队列是一种先进先出（FIFO, First In First Out）的结构。

> 队列是一种先进先出（FIFO）的数据结构，可以通过链表或数组实现。队列的基本操作包括创建队列、判空操作、入队操作和出队操作。队列常用于处理需要按照先后顺序进行操作的场景，例如消息队列、任务调度和广度优先搜索等。